<center><b><font color='red' size=5>Word Cloud For Teminologies</font></b></center>

---
### An Introduction To The Problem. "Background"
#### Given the subject book in plain-text form, we would like to get some terminologies out of the text book.
#### We are interested in the terminologies which are in `Chapter #1`.

---
## Table Of Contents:
1. #### Importing Necessary Packages. [Go to T.1](#T.1)
2. #### Formulate The Problem. [Go to T.2](#T.2)
3. #### Preprocessing The Text. [Go to T.3](#T.3)
4. #### Optional Step. [Go to T.4](#T.4)

---
<a id="T.1"></a>
### Importing Necessary Packages:
#### We May Need Some Of The Following Packages, And Modules: 
* ##### Pandas
* ##### Nltk
* ##### Seaborn, MatplotLib
* ##### Collections
* ##### Wordcloud

In [ ]:
# Import Necessary Modules And Packages.
import wordcloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import re

---
<a id="T.2"></a>
### Formulate The Problem:
#### In This Part, We Will Need To:
* ##### Read The Text The Correct Way.
* ##### Setup The Text For The Preprocessing Step.
* ##### Have A General Look At The Text To Gain Some Insights.
###### `PS:` To Get The Termonologies From The Desired Chapter, We Got Two Ways:
1. ###### Extract The Text Of Chapter #1 From The Whole Text Book.
2. ###### Extract The Section Of Terminologies In That Chapter From The Whole Book.

In [ ]:
# Read The Book Text File Using Pandas "read_csv" Function, With Determining The "delimiter" Parameter To Be "\t" Character.
text_book = pd.read_csv('Data mining.txt', delimiter='\t', header=None)
text_book.rename(columns = {0:'Text'}, inplace = True)
text_book

In [ ]:
# Get The Chapter 1 Text From The SBook.
ch1_mask = text_book['Text'].str.contains('1.1 What Is Business Analytics?')
ch1_index = list(text_book[ch1_mask].index)
print("Chapter 1 Occurences Indices:",ch1_index)

ch2_mask = text_book['Text'].str.contains('2.1 Introduction')
ch2_index = list(text_book[ch2_mask].index)
print("Chapter 2 Occurences Indices:",ch2_index)

chapter_1 = (text_book.iloc[ch1_index[1]:ch2_index[2]]).reset_index()
chapter_1.drop(['index'],axis=1,inplace=True)
chapter_1

In [ ]:
# Get The Terminology Section From Chapter 1 Text.
sect7_mask = text_book['Text'].str.contains('1.7 Terminology')
sect7_index = list(text_book[sect7_mask].index)
print("Chapter 1 Section 7 Occurences Indices:",sect7_index)

sect8_mask = text_book['Text'].str.contains('1.8 Road Maps')
sect8_index = list(text_book[sect8_mask].index)
print("Chapter 1 Section 8 Occurences Indices:",sect8_index)

terms_sect = (text_book.iloc[(sect7_index[1]+1):sect8_index[1]]).reset_index()
terms_sect.drop(['index'],axis=1,inplace=True)
terms_sect

---
<a id="T.3"></a>
### Preprocessing The Text.
#### In This Part, We Will Perform:
* ##### Remove Numbers From The Text Using `Regular Expressions`.
* ##### Remove Special Characters From The Text Using `Regular Expressions`.
* ##### Tokenize The Text `Split The Sentence\Text Into Individual Words List`.
* ##### Remove The Stop Words From The Text.
* ##### Generate A Word Cloud Of The Cleaned Text.

In [ ]:
# Preprocess The Text And Get A Word Cloud From It.
target_text = " ".join(chapter_1['Text'].values)
def preprocess_text(text_in):
    text_prep = re.sub(r'[0-9]', '', text_in)
    text_prep = re.sub(r'[^\w\s]', '', text_prep)
    tokens = word_tokenize(text_prep)
    stop_words = set(stopwords.words('english'))
    terms = [word for word in tokens if word.istitle() and len(word) > 3]
    terms_low = [word.lower() for word in terms]
    terms_low = [word for word in terms_low if word not in stop_words]
    return terms_low
text_prep = preprocess_text(target_text)
print(text_prep)

word_counts = Counter(text_prep)
wc = wordcloud.WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_counts)
plt.imshow(wc)
plt.axis('off')
plt.show()

In [ ]:
# Preprocess The Text And Get A Word Cloud From It.
target_text = " ".join(terms_sect['Text'].values)
text_prep = preprocess_text(target_text)
print(text_prep)

word_counts = Counter(text_prep)
wc = wordcloud.WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_counts)
plt.imshow(wc)
plt.axis('off')
plt.show()

---
<a id="T.4"></a>
### Optional Step.
#### In This Part, We Will Implement:
* ##### Gensim `Phrases` Model, To Evaluate Possible Collocations Among The Extracted Terminologies.

In [ ]:
# We Can Generate Collocations Using Gensim Model.
from gensim.models import Phrases
sentences_bytes = [[word.encode() for word in text_prep]]
n_gram_model = Phrases(sentences_bytes, min_count=5, delimiter=b'_')  
collocations = [key.decode() for key in n_gram_model.vocab.keys() if b'_' in key]
print(collocations)

In [ ]:
# Generate Word Frequencies From The Collocations, Then Get A Word Cloud From It.
word_freq = {}
for word in collocations:
    if word in word_freq:
        word_freq[word] += 1
    else:
        word_freq[word] = 1
wc = wordcloud.WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)
plt.imshow(wc) 
plt.axis("off") 
plt.show() 